<a href="https://colab.research.google.com/github/ihssane2002/sentiment-classification-project/blob/main/sentiment_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets


In [ ]:
!pip install --upgrade datasets fsspec huggingface_hub


In [ ]:
from datasets import load_dataset

ds = load_dataset("fancyzhx/amazon_polarity")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [ ]:
print(ds["train"][0])

{'label': 1, 'title': 'Stuning even for the non-gamer', 'content': 'This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'}


In [ ]:
import pandas as pd
from sklearn.utils import shuffle
from datasets import load_dataset, Dataset, DatasetDict

ds = load_dataset("fancyzhx/amazon_polarity")


train_df = ds['train'].shuffle(seed=42).select(range(5000)).to_pandas()
test_df = ds['test'].shuffle(seed=42).select(range(1000)).to_pandas()

# Séparer par classe (label 0 et 1)
positive_class_train = train_df[train_df['label'] == 1]
negative_class_train = train_df[train_df['label'] == 0]

positive_class_test = test_df[test_df['label'] == 1]
negative_class_test = test_df[test_df['label'] == 0]

# Équilibrer les classes
min_train = min(len(positive_class_train), len(negative_class_train))
min_test = min(len(positive_class_test), len(negative_class_test))

balanced_train = pd.concat([
    positive_class_train.sample(n=min_train, random_state=42),
    negative_class_train.sample(n=min_train, random_state=42)
])
balanced_test = pd.concat([
    positive_class_test.sample(n=min_test, random_state=42),
    negative_class_test.sample(n=min_test, random_state=42)
])

# Mélanger
balanced_train = shuffle(balanced_train, random_state=42).reset_index(drop=True)
balanced_test = shuffle(balanced_test, random_state=42).reset_index(drop=True)

# Convertir en datasets HuggingFace
ds = DatasetDict({
    'train': Dataset.from_pandas(balanced_train),
    'test': Dataset.from_pandas(balanced_test)
})

# Vérification des tailles et distribution
print("Taille du train :", len(ds['train']))
print(ds['train'][0])

print("Taille du test :", len(ds['test']))
print(ds['test'][0])


Taille du train : 4904
{'label': 0, 'title': 'Product ok, delivery just so-so', 'content': 'I ordered this product for my son\'s required reading. The price was very good but the shipping took the longest within the guidelines even though the seller stated that the book was available to ship out the same day, I received notice that it didn\'t ship out until many days later. When it finally arrived it was not in "good" condition; I would have rated it lower.'}
Taille du test : 974
{'label': 1, 'title': 'Great For kids who already know letters!', 'content': 'I am a nanny for two preschoolers: one 4.5 and one 5. They have letters and phonics down but making the transition to actually decoding words has not been easy. This video gives them a fun way to remember "how" to do it. It has great colors, music, and repetition that go along with fun ways to remember what a vowel is and even sound blends. I am very impressed with this...they are not big into TV/videos but they ask to watch this twi

**Fusionner les champs de texte**

In [ ]:
def combine_fields(example):
    example["text"] = example["title"] + " " + example["content"]
    return example

ds = ds.map(combine_fields)


Map:   0%|          | 0/4904 [00:00<?, ? examples/s]

Map:   0%|          | 0/974 [00:00<?, ? examples/s]

In [ ]:
from collections import Counter

print("Distribution des classes :", Counter(ds["train"]["label"]))


Distribution des classes : Counter({0: 2452, 1: 2452})


**Chargé le tokenizer**

In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
ds_split = ds["train"].train_test_split(test_size=0.2, seed=42)
ds = DatasetDict({
    "train": ds_split["train"],
    "validation": ds_split["test"],
    "test": ds["test"]
})


In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

tokenized_ds = ds.map(tokenize_function, batched=True)


Map:   0%|          | 0/3923 [00:00<?, ? examples/s]

Map:   0%|          | 0/981 [00:00<?, ? examples/s]

Map:   0%|          | 0/974 [00:00<?, ? examples/s]

In [ ]:
tokenized_ds = tokenized_ds.remove_columns(["title", "content", "text"])
tokenized_ds = tokenized_ds.rename_column("label", "labels")
tokenized_ds.set_format("torch")


In [ ]:
tokenized_ds["train"][0]


{'labels': tensor(0),
 'input_ids': tensor([    0, 10365,  5685,   572,   634,     5, 22395,    10,   367,   498,
             6,     8, 17741,     8, 18128,   148,   167,  5453,    71,  4835,
           112,    12,   176,  6696,     6,    38,  1276,     7,   860,    24,
             7,   192,   114,    24,    74, 10933,   143,  3766,     5,   511,
           662,    36,   698,   722,    71,   127,   200,  4049,     9,  1104,
          3984,     5, 25029,  1559,   322,    85,  1224,  2933,  2272,    12,
         22128,     6,  9172,    14,    38,    56,    10,  1233,  1280,     9,
          3766,    11,   127,  5803,     4,   440,   169,     4,    20,  9223,
           109,  8109,     5,  1296,    16, 43311, 43634,     6,  2128,   145,
          2132,   190,    30, 39616,   994,    11,     5,   935,   734,  4297,
           190,    11,     5,  5171,     9,  8143,   785,    50,    97,  1715,
             9,  3766,    11,     5,   443,     6,    24,   202,    16,   169,
           350,  

In [ ]:
from transformers import RobertaForSequenceClassification

# Charger le modèle RoBERTa pré-entraîné pour la classification binaire
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
display(tokenized_ds["train"].info)

DatasetInfo(description='', citation='', homepage='', license='', features={'labels': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}, post_processed=None, supervised_keys=None, builder_name=None, dataset_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

**Mettre les batchs sur le meme longuer**

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00


In [ ]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels)
    return { "accuracy": acc["accuracy"], "f1": f1_score["f1"] }


In [ ]:
!pip install --upgrade transformers


In [ ]:
import transformers
print(transformers.__version__)


4.51.3


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,  # Réduit
    per_device_eval_batch_size=8,
    num_train_epochs=5,  # Réduit pour tester
    learning_rate=3e-05,
    weight_decay=0.01,
    seed=42,
    logging_dir="./logs",
    logging_steps=20,
    warmup_steps=100,  # Réduit
    save_steps=1000,  # Moins fréquent
    save_total_limit=2,
    eval_steps=1000,  # Moins fréquent
    do_eval=True,
    do_train=True,
    lr_scheduler_type="linear",
    gradient_accumulation_steps=1,  # Simpler
)

print(" TrainingArguments configuré ")


 TrainingArguments configuré 


In [ ]:
import collections

# Convertir les tensors en valeurs et compter les occurrences de chaque label
labels = [label.item() for label in tokenized_ds["train"]["labels"]]

# Compter le nombre d'occurrences pour chaque label
label_counts = collections.Counter(labels)

# Afficher la distribution des labels
print(f"Distribution des labels dans le dataset d'entraînement : {label_counts}")


Distribution des labels dans le dataset d'entraînement : Counter({1: 1981, 0: 1942})


Sélectionner sous-ensemble de test et train

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingByLoss(threshold=0.07)],
)

# Entraînement
trainer.train()

# Évaluation
eval_results = trainer.evaluate()
print(eval_results)

<ipython-input-23-b0afdf8c30e0>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bammadihssane (bammadihssane-clup) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,0.698400
40,0.694000
60,0.680600
80,0.481400
100,0.307600
120,0.333000
140,0.297900
160,0.325800
180,0.374300
200,0.510500



✅ Loss de 0.0542 inférieure au seuil 0.07 — arrêt anticipé.


{'eval_loss': 0.20884792506694794, 'eval_accuracy': 0.9602446483180428, 'eval_f1': 0.959332638164755, 'eval_runtime': 14.1358, 'eval_samples_per_second': 69.398, 'eval_steps_per_second': 8.701, 'epoch': 2.079429735234216}


In [ ]:
from random import randint
import torch

# On choisit un exemple aléatoire du dataset original (non tokenisé)
sample = ds["test"][randint(0, len(ds["test"]) - 1)]

# Combiner "title" et "content" comme dans ta fonction `combine_fields`
text = sample["title"] + " " + sample["content"]

# Préparer les inputs
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Prédiction
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    prediction = outputs.logits.argmax(dim=1).item()

# Affichage
print("Texte :", text)
print("Vraie étiquette :", sample["label"])
print("Prédiction :", prediction)


Texte : Sedona Cliffs Notes An interesting book that pleasantly reminded me of my visits to Sedona and the people I met there. But aside from its nostalgic value, I found this book a bit choppy. A series of vignettes more than anything else, most without any significant story line development given the 3-5 pages allocated to each. One of these, a brief vignette of a secondary character being robbed on a cruise, has no relevance to the central plot, particularly given that the character is never addressed again in the book. There is a central story line, but it isn't very strong, and doesn't unfold very naturally. The book concludes in an open-ended way that may be thought-provoking, but it felt to me more like the author simply ran out of gas. While I finished this book, and did like it for its triggering memories of the area, I found it disappointing against the backdrop of the other highly favorable reviews posted here.
Vraie étiquette : 0
Prédiction : 0


In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Spécifie le chemin de sauvegarde dans ton Drive
save_path = "/content/drive/MyDrive/roberta-sentiment-model1"

# Sauvegarde du modèle et du tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Modèle et tokenizer sauvegardés dans : {save_path}")


Mounted at /content/drive
Modèle et tokenizer sauvegardés dans : /content/drive/MyDrive/roberta-sentiment-model1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Chemin du modèle dans Drive
model_path = "/content/drive/MyDrive/roberta-sentiment-model1"

# Charger le modèle et le tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)


Mounted at /content/drive


In [ ]:
print(model.config)


RobertaConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

